# Ejercicio a entregar

Cargue el fichero ejerciciop1.txt y cree un objeto pandas con cada línea del fichero. *Asegúrese que lo carga en la codificación de caracteres correcta (puede estar en utf-8 o latin)*

Añada una columna "symspellpy" con la corrección del diccionario en castellano "CREA_80000.txt" y otra columna "hunspell" con la corrección con el diccionario de HunSpell.

Por último guarde el CSV en formato utf-8.

Entregue este cuaderno de Jupyter y el fichero generado en la tarea correspondiente del Aulavirtual.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = "drive/MyDrive/cid-upct/Prácticas/3º Curso/2º Cuatrimestre/PNLE/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!sudo apt-get install libhunspell-dev
!pip install symspellpy
!pip install hunspell

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libhunspell-dev is already the newest version (1.7.0-4build1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [3]:
input_file = data_dir + "Datos/ejerciciop1.txt"
dict_path = data_dir + "Datos/CREA_80000.txt"
output_file = data_dir + "ejerciciop1_utf8.csv"  # Archivo de salida en UTF-8

# Cargar fichero en ejercicio p1
import pandas as pd
from symspellpy.symspellpy import SymSpell, Verbosity
import hunspell

# Iniciar SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary(dict_path, term_index=0, count_index=1)

# Iniciar HunSpell
hun_spell = hunspell.HunSpell(data_dir + "Datos/Spanish.dic", data_dir + "Datos/Spanish.aff")

# Leer el archivo línea por línea y almacenarlo en una lista
with open(input_file, "r", encoding="latin-1") as f:
    lineas = [line.strip() for line in f]

# Crear un DataFrame con la columna "text"
df = pd.DataFrame(lineas, columns=["text"])

# Usar sym_spell para las sugerencias de corrección compuesta
df["symspellpy"] = df['text'].apply(lambda x: sym_spell.lookup_compound(x, max_edit_distance=2))

# Función auxiliar que devuelve la primera sugerencia de HunSpell o el texto original
def get_first_suggestion(text):
    suggestions = hun_spell.suggest(text)
    return suggestions[0] if suggestions else text

df['hunspell'] = df['text'].apply(get_first_suggestion)

# Guardar el DataFrame en CSV con codificación UTF-8
df.to_csv(output_file, index=False, encoding="utf-8")

In [4]:
df = pd.read_csv(output_file)
df.head()

,text,symspellpy,hunspell
0,Ellos están felices porque mañana ban a la pla...,[<symspellpy.suggest_item.SuggestItem object a...,Ellos están felices porque mañana ban a la playa
1,Ayer vi a un amijo en el parke y charlamos un ...,[<symspellpy.suggest_item.SuggestItem object a...,Ayer vi a un amijo en el parque y charlamos un...
2,¿Por qué no me abisaste antes de salir de tú c...,[<symspellpy.suggest_item.SuggestItem object a...,¿Por qué no me abisaste antes de salir de tú c...
3,Los niños juegan en la calle sin abriggo aunqu...,[<symspellpy.suggest_item.SuggestItem object a...,Los niños juegan en la calle sin abriggo aunqu...
4,No se puede acer eso sin el permisso del profe...,[<symspellpy.suggest_item.SuggestItem object a...,No se puede acer eso sin el permisso del profe...
